In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Read in T1_manipulated df, casas df + estudio_trabajo df
T0 = pd.read_csv('/Users/katherinewu/Library/CloudStorage/GoogleDrive-kath.j.wu@gmail.com/My Drive/5. Coding/na-data-exploration/TrUST Data/T0_manipulated.csv', low_memory = False)
T1 = pd.read_csv('/Users/katherinewu/Library/CloudStorage/GoogleDrive-kath.j.wu@gmail.com/My Drive/5. Coding/na-data-exploration/TrUST Data/T1_manipulated.csv', low_memory = False)
T2 = pd.read_csv('/Users/katherinewu/Library/CloudStorage/GoogleDrive-kath.j.wu@gmail.com/My Drive/5. Coding/na-data-exploration/TrUST Data/T2_manipulated.csv', low_memory = False)
T3 = pd.read_csv('/Users/katherinewu/Library/CloudStorage/GoogleDrive-kath.j.wu@gmail.com/My Drive/5. Coding/na-data-exploration/TrUST Data/T3_manipulated.csv', low_memory = False)
casas = pd.read_csv('/Users/katherinewu/Library/CloudStorage/GoogleDrive-kath.j.wu@gmail.com/My Drive/5. Coding/na-data-exploration/TrUST Data/casa.csv', low_memory = False)
trabajo = pd.read_csv('/Users/katherinewu/Library/CloudStorage/GoogleDrive-kath.j.wu@gmail.com/My Drive/5. Coding/na-data-exploration/TrUST Data/estudio_trabajo.csv', low_memory = False)

In [ ]:
# Select only participants from Ciudad Bolivar in T0 + check length of df
T0_CB = T0[T0['localidad'] == 'Ciudad Bolivar']
T0_CB.head()
print(f"Total number of rows: {len(T0_CB)}") # Based on 'TrUST_T3 Sampling Strategy version larga', n = 1.031
print(f"Number of columns: {len(T0_CB.columns)}") # # of columns in 'Diccionario Encuesta Hogares' for T0-transmicable = 549


In [ ]:
# Select only participants from Ciudad Bolivar in casas + check length of df
casas_CB = casas[casas['transmi__3'] == 'Ciudad Bolivar']
casas_CB.head()
print(f"Total number of rows: {len(casas_CB)}") # This # matches T0
# Based on data exploration, n = 2052 for CB + SC

In [ ]:
# Select only participants from Ciudad Bolivar in T1 + check length of df
T1.head()
T1_CB = T1[T1['T1_localidad'] == 'Ciudad Bolivar'] 
print(f"Total number of rows: {len(T1_CB)}") # This # matches 'TrUST_T3 Sampling Strategy version larga', n = 825

In [ ]:
# Select only participants from Ciudad Bolivar in T2 + check length of df
T2.head()
T2_CB = T2[T2['T2_localidad'] == 1] # According 'Diccionario Encuesta Hogares' to 1 = CB 
#T2_CB = T2[T2['T2_Q3dir'] == 1] # Wanted to add question confirming that address stayed the same, but ended up filtering the df in a way that didn't match the sampling strategy #s
print(f"Total number of rows: {len(T2_CB)}") # this # matches 'TrUST_T3 Sampling Strategy version larga', n = 694

In [ ]:
# Select only participants from Ciudad Bolivar in T3 + check length of df
T3.head()
T3_CB = T3[T3['T3_ZONA_BASE'] == 'Ciudad Bolivar'] # According 'Diccionario Encuesta Hogares' to 1 = CB 
#T3_CB = T3[T3['T3_P3'] == 1] # Wanted to add question confirming that address stayed the same, but ended up filtering the df in a way that didn't match the sampling strategy #s
# Also tried to use 'T3_LOCALIDAD' == 19 for CB, but the length of the df also didn't match the sampling strategy #s
print(f"Total number of rows: {len(T3_CB)}") # this # matches 'TrUST_T3 Sampling Strategy version larga', n = 611

In [ ]:
# Test merging casas df with trabajo df to see how many participants we can match
c_t = pd.merge(casas, trabajo, left_on='transmi__1', right_on='Name', how='inner')
print(c_t.head())
print(f"Total number of rows: {len(c_t)}") # all participants matched

# Select columns of interest in new casas + trabajo df
c_t_select = c_t[['transmi__4', 'transmi__1', 'lat_x', 'lon_x', 'Name', 'lat_y', 'lon_y']].copy()
c_t_select.rename(columns={'transmi__4': 'Q2dir','transmi__1': 'casas_ID', 'Name': 'trabajo_ID'})
c_t_select.head()

In [ ]:

# Now that I've confirmed that the household survey #s for all time points (T0, T1, T2 + T3) closely align with 'TrUST_T3 Sampling Strategy version larga', I'll begin 
    # to merge them with the lat/long from the casas df - I need to use T0 bc it's the only df with addresses included (Q2dir), which is what I'll merge on
T0_c_merged = pd.merge(T0_CB, casas_CB, left_on='Q2dir', right_on='transmi__4', how='inner')
print(T0_c_merged.head())
print(f"Total number of rows: {len(T0_c_merged)}") # 878 vs. 877 when done with the entire df not isolated for CB locality

In [ ]:
# Isolate df down to the most basic items of interest to test #s
T0_c = T0_c_merged[['ID', 'transmi__1', 'localidad', 'transmi__3', 'Q2dir', 'transmi__4', 'lat', 'lon', 'Q2dir', 'Q11', 'Q12', 'Q13', 'Q15', 'Q16', 'Q19', 'Q87c1', 'Q115', 'Q116']]

In [ ]:
T0_c.head()

In [ ]:
T3_CB_select = T3_CB[['T3_ID_INF', 'T3_ZONA_BASE']]
T3_CB_select.head()

In [ ]:
# Test merging T0_c merged df on T3 to see how many participants we can match in the final timeframe
T03_c = pd.merge(T0_c, T3_CB_select, left_on='ID', right_on='T3_ID_INF', how='inner')
print(T03_c.head())
print(f"Total number of rows: {len(T03_c)}") # sample size in sampling strategy is n = 598

In [ ]:
T2_CB.head()

In [ ]:
T2_CB_select = T2_CB[['T2_ID', 'T2_localidad']]
T2_CB_select.head()

In [ ]:
# Test merging T03_c merged df with T2 to see how many participants we can match
T023_c = pd.merge(T03_c, T2_CB_select, left_on='ID', right_on='T2_ID', how='inner')
print(T023_c.head())
print(f"Total number of rows: {len(T023_c)}")

In [ ]:
T1_CB.head() # look at ID11060170121129 -- address changed but remained within CB, I'm most likely missing a few of these + need to decide whether I keep only the participants that
    # remained in the same location for all 4 time periods

In [ ]:
T1_CB_select = T1_CB[['T1_ID', 'T1_localidad', 'T1_Q2dir']]
T1_CB_select.head()

In [ ]:
# Test merging T023_c merged df with T1 to see how many participants we can match
T0123_c = pd.merge(T023_c, T1_CB_select, left_on='ID', right_on='T1_ID', how='inner')
print(T0123_c.head())
print(f"Total number of rows: {len(T0123_c)}") # n = 515, no loss between T023_c + T0123_c

In [ ]:
# Save T0123_c df to csv to ingest into QGIS
#T0123_c.to_csv('/Users/katherinewu/Library/CloudStorage/GoogleDrive-kath.j.wu@gmail.com/My Drive/5. Coding/na-data-exploration/TrUST Data/T0123_c.csv', index=False)

In [ ]:
print(trabajo.head())

In [ ]:
# Change type of ID columns for casas + trabajo to integer to merge
casas['transmi__1'] = casas['transmi__1'].astype(int)
trabajo['Name'] = trabajo['Name'].astype(int)

In [ ]:
# Test merging casas df with trabajo df to see how many participants we can match
c_t = pd.merge(casas, trabajo, left_on='transmi__1', right_on='Name', how='inner')
print(c_t.head())
print(f"Total number of rows: {len(c_t)}") # all participants matched

In [ ]:
# Merge casas and trabajo
c_t = pd.merge(casas, trabajo, left_on='transmi__1', right_on='Name', how='inner')
print(c_t.head())
print(f"Total number of rows: {len(c_t)}")  # all participants matched

# Select and rename columns
c_t_select = c_t[['transmi__1', 'lat_x', 'lon_x', 'Name', 'lat_y', 'lon_y']].copy()
c_t_select = c_t_select.rename(columns={
    'transmi__1': 'casas_ID',
    'Name': 'trabajo_ID'
})

print(c_t_select.head())

In [ ]:
T0123_c['transmi__1'] = T0123_c['transmi__1'].astype(int)
c_t_select.loc[:, 'casas_ID'] = c_t_select['casas_ID'].astype(int)

In [ ]:
# Will now combine T0123 df wih trabajo date to add in each participant's work location
T0123_ct = pd.merge(T0123_c, c_t, left_on='transmi__1', right_on='transmi__1', how='inner') # Refer to email from MAW on dataset details
T0123_ct.head()
print(f"Total number of rows: {len(T0123_ct)}") # all participants matched

In [ ]:
T0123_ct

In [ ]:
T0123_ct.head()
len(T0123_ct)

In [ ]:
len(T0123_c)

In [ ]:
column_values_ct = T0123_ct['transmi__1'].tolist()
print(column_values_ct)

In [ ]:
column_values_c = T0123_c['transmi__1'].tolist()
print(column_values_c)

In [ ]:
column_values_ct_sel = c_t_select['casas_ID'].tolist()
print(column_values_ct_sel)

In [ ]:
# Find common values
common_values = set(c_t_select['casas_ID']).intersection(set(T0123_c['transmi__1']))

# Count the number of common values
common_count = len(common_values)

#print(f"Common values: {common_values}")
print(f"Number of common values: {common_count}")

In [ ]:
# Find common values
common_values = set(trabajo['Name']).intersection(set(T0123_c['transmi__1']))

# Count the number of common values
common_count = len(common_values)

#print(f"Common values: {common_values}")
print(f"Number of common values: {common_count}")

In [ ]:
# Find common values
common_values = set(trabajo['Name']).intersection(set(casas['transmi__1']))

# Count the number of common values
common_count = len(common_values)

#print(f"Common values: {common_values}")
print(f"Number of common values: {common_count}")

In [ ]:
# Find common values
common_values = set(casas['transmi__1']).intersection(set(T0123_c['transmi__1']))

# Count the number of common values
common_count = len(common_values)

#print(f"Common values: {common_values}")
print(f"Number of common values: {common_count}")

In [ ]:
trabajo.dtypes

In [ ]:
casas.dtypes

In [ ]:
# Convert columns to sets
set_df1 = set(T0123_c['transmi__1'])
set_df2 = set(c_t_select['casas_ID'])

# Find non-matching values from df1
non_matching_from_df1 = list(set_df1 - set_df2)

# Find non-matching values from df2
non_matching_from_df2 = list(set_df2 - set_df1)

print(f"Non-matching values from df1: {non_matching_from_df1}")
print(f"Non-matching values from df2: {non_matching_from_df2}")

In [ ]:
# Convert columns to sets
set_df1 = set(T0123_c['transmi__1'])
set_df2 = set(casas_CB['transmi__1'])

# Find non-matching values from df1
non_matching_from_df1 = list(set_df1 - set_df2)

# Find non-matching values from df2
non_matching_from_df2 = list(set_df2 - set_df1)

print(f"Non-matching values from df1: {non_matching_from_df1}")
print(f"Non-matching values from df2: {non_matching_from_df2}")
print(len(non_matching_from_df2))
print(len(non_matching_from_df1))

In [ ]:
print(len(casas)-len(casas_CB))
print(len(casas_CB))
print(len(casas))
print(len(casas_CB)-len(non_matching_from_df2))

In [ ]:
c_t_select.head()

In [ ]:
print(len(casas))
casas.head()

In [ ]:
print(T0123_c.columns)
print(c_t_select.columns)

In [ ]:
# Create new data frame from T0123_c
T0123_c_clean = T0123_c[['ID', 'transmi__1', 'localidad', 'transmi__3', 'transmi__4', 'lat', 'lon', 'Q11', 'Q12', 'Q13', 'Q15', 'Q16', 'Q19', 'Q87c1', 'Q115', 'Q116']]
T0123_c_clean.head()

In [ ]:
# Save c_t df to csv to compare outside of VSCode
#c_t.to_csv('/Users/katherinewu/Library/CloudStorage/GoogleDrive-kath.j.wu@gmail.com/My Drive/5. Coding/na-data-exploration/TrUST Data/c_t.csv', index=False)

In [ ]:
# Save T0123_ct df to csv to ingest into QGIS
#T0123_ct.to_csv('/Users/katherinewu/Library/CloudStorage/GoogleDrive-kath.j.wu@gmail.com/My Drive/5. Coding/na-data-exploration/TrUST Data/T0123_ct.csv', index=False)